# LIBRERIAS

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import random
import matplotlib.pyplot as plt
import math
import itertools
import time

# METODOS

## ALGORITMO GENÉTICO

In [ ]:
def calcular_distancia_gen(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def calcular_longitud_gen(ruta, puntos):
    longitud_total = 0
    for i in range(len(ruta) - 1):
        longitud_total += calcular_distancia_gen(puntos[ruta[i]], puntos[ruta[i + 1]])
    longitud_total += calcular_distancia_gen(puntos[ruta[-1]], puntos[ruta[0]])
    return longitud_total

def generar_poblacion_gen(tamano, num_puntos):
    poblacion = []
    for _ in range(tamano):
        individuo = list(range(num_puntos))
        random.shuffle(individuo)
        poblacion.append(individuo)
    return poblacion

def seleccion_gen(poblacion, puntos):
    puntuaciones = []
    for individuo in poblacion:
        puntuaciones.append(1 / calcular_longitud_gen(individuo, puntos))
    total_puntuacion = sum(puntuaciones)
    probabilidades = [p / total_puntuacion for p in puntuaciones]

    elegido = random.choices(poblacion, probabilidades, k=2)
    return elegido

def cruce_gen(padre1, padre2):
    start, end = sorted(random.sample(range(len(padre1)), 2))
    hijo = [-1] * len(padre1)
    hijo[start:end+1] = padre1[start:end+1]
    curr = 0
    for i in range(len(padre2)):
        if padre2[i] not in hijo:
            while hijo[curr] != -1:
                curr += 1
            hijo[curr] = padre2[i]
    return hijo

def mutacion_gen(individuo, tasa_mutacion_gen):
    if random.random() < tasa_mutacion_gen:
        i, j = random.sample(range(len(individuo)), 2)
        individuo[i], individuo[j] = individuo[j], individuo[i]
    return individuo

def algoritmo_genetico_gen(puntos, tamano_poblacion=100, generaciones=100, tasa_mutacion_gen=0.05):
    num_puntos = len(puntos)
    poblacion = generar_poblacion_gen(tamano_poblacion, num_puntos)

    mejor_ruta = None
    mejor_longitud = float('inf')

    for generacion in range(generaciones):
        nueva_poblacion = []

        for _ in range(tamano_poblacion // 2):
            padre1, padre2 = seleccion_gen(poblacion, puntos)
            hijo1 = cruce_gen(padre1, padre2)
            hijo2 = cruce_gen(padre2, padre1)
            nueva_poblacion.append(mutacion_gen(hijo1, tasa_mutacion_gen))
            nueva_poblacion.append(mutacion_gen(hijo2, tasa_mutacion_gen))

        poblacion = nueva_poblacion

        for individuo in poblacion:
            longitud = calcular_longitud_gen(individuo, puntos)
            if longitud < mejor_longitud:
                mejor_longitud = longitud
                mejor_ruta = individuo

    return mejor_ruta, mejor_longitud

def graficar_ruta_gen(puntos, ruta):
    x = [puntos[i][0] for i in ruta] + [puntos[ruta[0]][0]]
    y = [puntos[i][1] for i in ruta] + [puntos[ruta[0]][1]]

    plt.figure(figsize=(8, 6))
    plt.plot(x, y, marker='o', linestyle='-', color='b', label='Ruta óptima')
    plt.scatter([p[0] for p in puntos], [p[1] for p in puntos], color='r', zorder=5)

    for i, point in enumerate(puntos):
        plt.text(point[0], point[1], f'  {i}', fontsize=12)

    plt.title('Problema del Agente Viajero - Algoritmo Genético')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.legend()
    plt.show()

## ALGORITMO GREED

In [ ]:
def calcular_distancia_greed(ciudad1, ciudad2):
    return math.sqrt((ciudad2[0] - ciudad1[0])**2 + (ciudad2[1] - ciudad1[1])**2)

def greedy_tsp_greed(coordenadas):
    n = len(coordenadas)
    visitadas = [False] * n
    recorrido = []
    ciudad_actual = 0
    visitadas[ciudad_actual] = True
    recorrido.append(ciudad_actual)

    while len(recorrido) < n:
        siguiente_ciudad = None
        min_distancia = float('inf')

        for i in range(n):
            if not visitadas[i]:
                distancia = calcular_distancia_greed(coordenadas[ciudad_actual], coordenadas[i])
                if distancia < min_distancia:
                    siguiente_ciudad = i
                    min_distancia = distancia

        visitadas[siguiente_ciudad] = True
        recorrido.append(siguiente_ciudad)
        ciudad_actual = siguiente_ciudad

    # Regresa al punto de inicio para completar el ciclo
    recorrido.append(recorrido[0])
    return recorrido

def calcular_distancia_greed_total(coordenadas, recorrido):
    distancia_total = 0
    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        distancia_total += calcular_distancia_greed(coordenadas[ciudad_actual], coordenadas[ciudad_siguiente])
    return distancia_total

def graficar_ruta_greed(puntos, recorrido):
    x = [puntos[i][0] for i in recorrido] + [puntos[recorrido[0]][0]]
    y = [puntos[i][1] for i in recorrido] + [puntos[recorrido[0]][1]]

    plt.figure(figsize=(8, 6))
    plt.plot(x, y, marker='o', linestyle='-', color='b', label='Ruta óptima')
    plt.scatter([p[0] for p in puntos], [p[1] for p in puntos], color='r', zorder=5)

    for i, point in enumerate(puntos):
        plt.text(point[0], point[1], f'  {i}', fontsize=12)

    plt.title('Problema del Agente Viajero - Algoritmo Greed')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.legend()
    plt.show()

## ALGORITMO BUSQUEDA LOCAL

In [ ]:
def calcular_distancia_local(ciudad1, ciudad2):
    return math.sqrt((ciudad2[0] - ciudad1[0])**2 + (ciudad2[1] - ciudad1[1])**2)

def calcular_distancia_total_local(coordenadas, recorrido):
    distancia_total = 0
    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        distancia_total += calcular_distancia_local(coordenadas[ciudad_actual], coordenadas[ciudad_siguiente])

    ciudad_actual = recorrido[-1]
    ciudad_siguiente = recorrido[0]
    distancia_total += calcular_distancia_local(coordenadas[ciudad_actual], coordenadas[ciudad_siguiente])
    return distancia_total

def two_opt_local(coordenadas, recorrido):
    n = len(coordenadas)
    mejor_recorrido = recorrido[:]
    mejor_distancia = calcular_distancia_total_local(coordenadas, mejor_recorrido)

    mejorado = True
    while mejorado:
        mejorado = False
        for i in range(1, n - 2):
            for j in range(i + 1, n):
                if j - i == 1:
                    continue

                nuevo_recorrido = mejor_recorrido[:]

                nuevo_recorrido[i:j] = reversed(nuevo_recorrido[i:j])

                nueva_distancia = calcular_distancia_total_local(coordenadas, nuevo_recorrido)

                if nueva_distancia < mejor_distancia:
                    mejor_recorrido = nuevo_recorrido[:]
                    mejor_distancia = nueva_distancia
                    mejorado = True
                    break
            if mejorado:
                break
    return mejor_recorrido

def graficar_recorrido_local(coordenadas, recorrido):
    x = [coordenadas[i][0] for i in recorrido]
    y = [coordenadas[i][1] for i in recorrido]

    plt.figure(figsize=(8, 6))

    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        plt.plot([coordenadas[ciudad_actual][0], coordenadas[ciudad_siguiente][0]],
                 [coordenadas[ciudad_actual][1], coordenadas[ciudad_siguiente][1]],
                 'bo-', markersize=8)

    plt.plot([coordenadas[recorrido[-1]][0], coordenadas[recorrido[0]][0]],
             [coordenadas[recorrido[-1]][1], coordenadas[recorrido[0]][1]],
             'bo-', markersize=8)

    for i, (xi, yi) in enumerate(coordenadas):
        plt.text(xi, yi, f' {i+1}', fontsize=12, ha='right', color='red',label='Ruta óptima')

    plt.title("Problema del Agente Viajero - Busqueda Local")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.show()

def generar_recorrido_inicial_local(n):
    return random.sample(range(n), n)

## ALGORITMO FUERZA BRUTA

In [ ]:
def calcular_distancia_fuerza(ciudad1, ciudad2):
    return math.sqrt((ciudad2[0] - ciudad1[0])**2 + (ciudad2[1] - ciudad1[1])**2)

def tcalcular_distancia_total_fuerza(coordenadas, recorrido):
    distancia_total = 0
    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        distancia_total += calcular_distancia_fuerza(coordenadas[ciudad_actual], coordenadas[ciudad_siguiente])

    ciudad_actual = recorrido[-1]
    ciudad_siguiente = recorrido[0]
    distancia_total += calcular_distancia_fuerza(coordenadas[ciudad_actual], coordenadas[ciudad_siguiente])
    return distancia_total

def graficar_recorrido_fuerza(coordenadas, recorrido):

    x = [coordenadas[i][0] for i in recorrido]
    y = [coordenadas[i][1] for i in recorrido]

    plt.figure(figsize=(8, 6))

    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        plt.plot([coordenadas[ciudad_actual][0], coordenadas[ciudad_siguiente][0]],
                 [coordenadas[ciudad_actual][1], coordenadas[ciudad_siguiente][1]],
                 'bo-', markersize=8)

    plt.plot([coordenadas[recorrido[-1]][0], coordenadas[recorrido[0]][0]],
             [coordenadas[recorrido[-1]][1], coordenadas[recorrido[0]][1]],
             'bo-', markersize=8)

    for i, (xi, yi) in enumerate(coordenadas):
        plt.text(xi, yi, f'{i+1}', fontsize=12, ha='right', color='red' ,label='Ruta óptima')

    plt.title("Problema del Agente Viajero – Fuerza Bruta")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.show()

def fuerza_bruta_fuerza(coordenadas):

    ciudades = list(range(len(coordenadas)))
    permutaciones = itertools.permutations(ciudades)

    mejor_recorrido = None
    mejor_distancia = float('inf')

    for recorrido in permutaciones:
        distancia = tcalcular_distancia_total_fuerza(coordenadas, recorrido)
        if distancia < mejor_distancia:
            mejor_distancia = distancia
            mejor_recorrido = recorrido

    return mejor_recorrido, mejor_distancia

##  ALGORITMO COLONIA DE HORMIGA

In [ ]:
def calcular_distancia_hormiga(ciudad1, ciudad2):
    return math.sqrt((ciudad2[0] - ciudad1[0])**2 + (ciudad2[1] - ciudad1[1])**2)

def calcular_costo_hormiga(recorrido, distancias):
    costo = 0
    for i in range(len(recorrido) - 1):
        costo += distancias[recorrido[i]][recorrido[i + 1]]
    costo += distancias[recorrido[-1]][recorrido[0]]  # Volver al inicio
    return costo

# Algoritmo de Colonia de Hormigas para TSP
def colonia_hormigas(coordenadas, num_hormigas, num_iteraciones, alfa=1.0, beta=2.0, rho=0.1, Q=100):
    n = len(coordenadas)

    # Crear la matriz de distancias entre las ciudades
    distancias = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distancias[i][j] = calcular_distancia_hormiga(coordenadas[i], coordenadas[j])

    # Inicializar feromonas
    feromonas = np.ones((n, n))  # Inicializamos con un valor pequeño
    mejor_recorrido = None
    mejor_costo = float('inf')

    # Bucle principal del algoritmo
    for _ in range(num_iteraciones):
        recorridos = []
        costos = []

        # Las hormigas construyen su recorrido
        for _ in range(num_hormigas):
            recorrido = construir_recorrido_hormiga(feromonas, distancias, alfa, beta)
            costo = calcular_costo_hormiga(recorrido, distancias)
            recorridos.append(recorrido)
            costos.append(costo)

            # Actualizar la mejor solución encontrada
            if costo < mejor_costo:
                mejor_costo = costo
                mejor_recorrido = recorrido

        # Actualizar feromonas
        feromonas = (1 - rho) * feromonas  # Evaporación de feromonas
        for i in range(num_hormigas):
            for j in range(len(recorridos[i]) - 1):
                feromonas[recorridos[i][j]][recorridos[i][j + 1]] += Q / costos[i]  # Depósito de feromonas

        # Mostrar progreso (opcional)
        #print(f"Iteración: {_+1}, Mejor costo: {mejor_costo:.2f}")

    return mejor_recorrido, mejor_costo

# Función para que una hormiga construya su recorrido
def construir_recorrido_hormiga(feromonas, distancias, alfa, beta):
    n = len(distancias)
    recorrido = [0]
    no_visitadas = set(range(1, n))  # Ciudades no visitadas

    while no_visitadas:
        ciudad_actual = recorrido[-1]

        # Calcular probabilidades de elegir las siguientes ciudades
        probabilidades = []
        total = 0
        for ciudad in no_visitadas:
            probabilidad = (feromonas[ciudad_actual][ciudad] ** alfa) * ((1.0 / distancias[ciudad_actual][ciudad]) ** beta)
            probabilidades.append(probabilidad)
            total += probabilidad

        # Normalizar las probabilidades
        probabilidades = [p / total for p in probabilidades]

        # Elegir la siguiente ciudad según las probabilidades
        siguiente_ciudad = random.choices(list(no_visitadas), weights=probabilidades, k=1)[0]
        recorrido.append(siguiente_ciudad)
        no_visitadas.remove(siguiente_ciudad)

    return recorrido

def graficar_recorrido_hormiga(coordenadas, recorrido):
    x = [coordenadas[i][0] for i in recorrido]
    y = [coordenadas[i][1] for i in recorrido]
    plt.figure(figsize=(8, 6))

    # Conectar las ciudades en el recorrido optimizado
    for i in range(len(recorrido) - 1):
        ciudad_actual = recorrido[i]
        ciudad_siguiente = recorrido[i + 1]
        plt.plot([coordenadas[ciudad_actual][0], coordenadas[ciudad_siguiente][0]],
                 [coordenadas[ciudad_actual][1], coordenadas[ciudad_siguiente][1]],
                 'bo-', markersize=8)

    # Conectar la última ciudad con la primera para completar el ciclo
    plt.plot([coordenadas[recorrido[-1]][0], coordenadas[recorrido[0]][0]],
             [coordenadas[recorrido[-1]][1], coordenadas[recorrido[0]][1]],
             'bo-', markersize=8)

    for i, (xi, yi) in enumerate(coordenadas):
        plt.text(xi, yi, f'{i+1}', fontsize=12, ha='right', color='red',label='Ruta óptima')

    plt.title("Problema del Agente Viajero – Colonia de Hormigas")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.grid(True)
    plt.show()

## TABLA COMPARATIVA

In [ ]:
def reemplazos(variable):
  variable=str(variable)
  variable=variable.replace('[','')
  variable=variable.replace("]",'')
  variable=variable.replace('(','')
  variable=variable.replace(')','')
  return variable

In [ ]:
def generar_pares_coordenados(n):
    pares = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(n)]
    return pares

In [ ]:
def tabla(lista):

  # ALGORITMO GENETICO
  inicio=time.time()
  TRAYECTO_GEN, DIST_GEN = algoritmo_genetico_gen(lista)
  fin=time.time()
  TRAYECTO_GEN=reemplazos(TRAYECTO_GEN)
  TIEMPO_GEN=fin-inicio

  # ALGORITMO GREED
  inicio=time.time()
  TRAYECTO_GREED = greedy_tsp_greed(lista)
  DIST_GREED=calcular_distancia_greed_total(lista, TRAYECTO_GREED)
  TRAYECTO_GREED=reemplazos(TRAYECTO_GREED)
  fin=time.time()
  TIEMPO_GREED=fin-inicio

  # ALGORITMO BUSQUEDA LOCAL
  inicio=time.time()
  n=len(lista)
  recorrido_inicial = generar_recorrido_inicial_local(n)
  distancia_inicial = calcular_distancia_total_local(lista, recorrido_inicial)
  TRAYECTO_LOCAL = two_opt_local(lista, recorrido_inicial)
  DIST_LOCAL = calcular_distancia_total_local(lista, TRAYECTO_LOCAL)

  TRAYECTO_LOCAL=reemplazos(TRAYECTO_LOCAL)
  fin=time.time()
  TIEMPO_LOCAL=fin-inicio

  # ALGORITMO FUERZA BRUTA
  inicio=time.time()
  TRAYECTO_FUERZA, DIST_FUERZA = fuerza_bruta_fuerza(lista)

  TRAYECTO_FUERZA=reemplazos(TRAYECTO_FUERZA)
  fin=time.time()
  TIEMPO_FUERZA=fin-inicio

  # ALGORITMO COLONIA DE HORMIGAS
  inicio=time.time()
  num_hormigas = 10
  num_iteraciones = 100
  alfa = 1.0     # Influencia de las feromonas
  beta = 2.0     # Influencia de la distancia
  rho = 0.1      # Tasa de evaporación
  Q = 100        # Depósito de feromonas
  TRAYECTO_HORMIGA, DIST_HORMIGA = colonia_hormigas(lista, num_hormigas, num_iteraciones, alfa, beta, rho, Q)

  TRAYECTO_HORMIGA=reemplazos(TRAYECTO_HORMIGA)
  fin=time.time()
  TIEMPO_HORMIGA=fin-inicio

  RESUMEN=pd.DataFrame({'ALGORITMO':['ALGORITMO GENÉTICO','ALGORITMO GREED','ALGORITMO DE BUSQUEDA LOCAL','ALGORITMO DE FUERZA BRUTA','ALGORITMO DE COLONIA DE HORMIGAS'],
                                                  'TIEMPO (SEGUNDOS) ':[TIEMPO_GEN,TIEMPO_GREED,TIEMPO_LOCAL,TIEMPO_FUERZA,TIEMPO_HORMIGA],
                                                  'RECORRIDO ÓPTIMO':[TRAYECTO_GEN,TRAYECTO_GREED,TRAYECTO_LOCAL,TRAYECTO_FUERZA,TRAYECTO_HORMIGA],
                                                  'DISTANCIA ÓPTIMA':[DIST_GEN,DIST_GREED,DIST_LOCAL,DIST_FUERZA,DIST_HORMIGA] })
  return RESUMEN

# RESULTADOS

In [ ]:
NUMEROS=[3,4,5,6,7,8,9,10,11,12]
TABLA_BASE_DISTANCIA=pd.DataFrame({'ALGORITMO':['ALGORITMO GENÉTICO','ALGORITMO GREED','ALGORITMO DE BUSQUEDA LOCAL','ALGORITMO DE FUERZA BRUTA','ALGORITMO DE COLONIA DE HORMIGAS']})
TABLA_BASE_TIEMPO=pd.DataFrame({'ALGORITMO':['ALGORITMO GENÉTICO','ALGORITMO GREED','ALGORITMO DE BUSQUEDA LOCAL','ALGORITMO DE FUERZA BRUTA','ALGORITMO DE COLONIA DE HORMIGAS']})
for n in NUMEROS:
  print(n)
  puntos=generar_pares_coordenados(n)
  TEMPORAL=tabla(puntos)
  TEMPORAL=TEMPORAL.drop('RECORRIDO ÓPTIMO',axis=1)
  tiempo='TIEMPO PARA '+str(n)+' PUNTOS'
  distancia='DITANCIA ÓPTIMA '+str(n)+' PUNTOS'
  TEMPORAL=TEMPORAL.rename(columns={'TIEMPO (SEGUNDOS) ':tiempo,'DISTANCIA ÓPTIMA':distancia})
  TABLA_BASE_DISTANCIA=TABLA_BASE_DISTANCIA.merge(TEMPORAL[['ALGORITMO',distancia]],on='ALGORITMO',how='left')
  TABLA_BASE_TIEMPO=TABLA_BASE_TIEMPO.merge(TEMPORAL[['ALGORITMO',tiempo]],on='ALGORITMO',how='left')

3
4
5
6
7
8
9
10
11
12


In [ ]:
TABLA_BASE_DISTANCIA

,ALGORITMO,DITANCIA ÓPTIMA 3 PUNTOS,DITANCIA ÓPTIMA 4 PUNTOS,DITANCIA ÓPTIMA 5 PUNTOS,DITANCIA ÓPTIMA 6 PUNTOS,DITANCIA ÓPTIMA 7 PUNTOS,DITANCIA ÓPTIMA 8 PUNTOS,DITANCIA ÓPTIMA 9 PUNTOS,DITANCIA ÓPTIMA 10 PUNTOS,DITANCIA ÓPTIMA 11 PUNTOS,DITANCIA ÓPTIMA 12 PUNTOS
0,ALGORITMO GENÉTICO,141.779273,198.451843,226.072472,256.293453,272.584504,266.326606,305.356751,287.999483,367.458415,400.042778
1,ALGORITMO GREED,141.779273,199.600586,226.072472,256.293453,273.625795,326.683472,349.217272,290.779276,352.639142,484.993891
2,ALGORITMO DE BUSQUEDA LOCAL,141.779273,198.451843,252.912961,286.151195,300.045414,300.695465,353.873716,325.752295,330.267001,386.496158
3,ALGORITMO DE FUERZA BRUTA,141.779273,198.451843,226.072472,256.293453,272.584504,266.326606,298.045712,275.864782,330.267001,362.728238
4,ALGORITMO DE COLONIA DE HORMIGAS,141.779273,198.451843,226.072472,256.293453,272.584504,266.326606,299.179695,275.864782,330.267001,362.728238


In [ ]:
TABLA_BASE_TIEMPO

,ALGORITMO,TIEMPO PARA 3 PUNTOS,TIEMPO PARA 4 PUNTOS,TIEMPO PARA 5 PUNTOS,TIEMPO PARA 6 PUNTOS,TIEMPO PARA 7 PUNTOS,TIEMPO PARA 8 PUNTOS,TIEMPO PARA 9 PUNTOS,TIEMPO PARA 10 PUNTOS,TIEMPO PARA 11 PUNTOS,TIEMPO PARA 12 PUNTOS
0,ALGORITMO GENÉTICO,26.228023,13.515684,16.641295,20.764627,23.741634,27.417518,30.899687,32.417967,36.780559,39.545808
1,ALGORITMO GREED,0.000024,0.000031,0.000037,0.000044,0.000062,0.000062,0.000087,0.000079,0.000093,0.000111
2,ALGORITMO DE BUSQUEDA LOCAL,0.000026,0.000048,0.000097,0.000106,0.000393,0.000690,0.000611,0.001424,0.001155,0.004048
3,ALGORITMO DE FUERZA BRUTA,0.000038,0.000119,0.000662,0.004538,0.048193,0.340499,4.596375,41.264160,507.966452,6591.550188
4,ALGORITMO DE COLONIA DE HORMIGAS,0.013513,0.021657,0.037229,0.041078,0.053933,0.067192,0.080013,0.095167,0.111469,0.125986
